In [1]:
#librerias
import numpy as np
import pandas as pd 
import modulo_conn_sql as mcq
import pyodbc
import sqlalchemy as sa
import urllib

def conectarSQL():
    conn = mcq.ConexionSQL()
    cursor = conn.getCursor()
    return cursor

#Query BD SQL-Server Cemex
def querySQL(query, parametros):
    #Conectar con base sql y ejecutar consulta
    cursor = conectarSQL()
    try:
        cursor.execute(query, parametros)
        #obtener nombre de columnas
        names = [ x[0] for x in cursor.description]
        
        #Reunir todos los resultado en rows
        rows = cursor.fetchall()
        resultadoSQL = []
            
        #Hacer un array con los resultados
        while rows:
            resultadoSQL.append(rows)
            if cursor.nextset():
                rows = cursor.fetchall()
            else:
                rows = None
                
        #Redimensionar el array para que quede en dos dimensiones
        resultadoSQL = np.array(resultadoSQL)
        resultadoSQL = np.reshape(resultadoSQL, (resultadoSQL.shape[1], resultadoSQL.shape[2]) )
    finally:
            if cursor is not None:
                cursor.close()
    return pd.DataFrame(resultadoSQL, columns = names)
 

<h1> VENTANA HORARIA </h1>

In [4]:
def calcular_ventana(yearTarget, entresemana, meses ):

    ventana = pd.DataFrame()
    #ventana entre semana
    if entresemana == 1:
        inicio_semana = 2
        fin_semana = 6
    #ventana fin de semana
    elif entresemana == 0:
        inicio_semana = 7
        fin_semana = 7
    
    for j in meses:
        mesTarget = j + 1
        if j == 0:
            ventana = querySQL( "{CALL SCAC_AP16_calculo_ventana_horaria_equivalente (?,?,?,?)}" , (yearTarget, mesTarget, inicio_semana, fin_semana ) )
        else:
            temp = querySQL( "{CALL SCAC_AP16_calculo_ventana_horaria_equivalente (?,?,?,?)}" , (yearTarget, mesTarget, inicio_semana, fin_semana ) )
            ventana = pd.concat([ventana, temp])
    ventana['entresemana'] = entresemana
    return ventana

In [5]:
years = [2021]
meses = [3] # ene = 0, feb = 1, ... dic=11

for k in years:
    df_ventana = calcular_ventana(k, 1, meses)

df_ventana

,año,mes,Planta,ventanaHoraria,entresemana
0,2021,4,CO-PLANTA CALI,5.9930279153300745,1
1,2021,4,CO-PLANTA PAMPLONA,9.57710843373494,1
2,2021,4,DO-PLANTA MARAÑON,7.420940253145078,1
3,2021,4,PR-PLANTA PONCE II,7.759788780377015,1
4,2021,4,CO-PLANTA FLORIDA,5.579256172104469,1
5,2021,4,CO-PLANTA NEIVA,7.613995693397433,1
6,2021,4,CO-PLANTA PEREIRA,11.211392929659185,1
7,2021,4,CO-PLANTA SANTA MARTA,5.882000367908085,1
8,2021,4,DO-PLANTA BARRICK,12.047808764940235,1
9,2021,4,DO-PLANTA LOS PINOS,6.898386144126393,1


In [6]:
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=COOCCAPP11A;DATABASE=BI_Tableau;UID=usertableau;PWD=usertableau$")
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

df_ventana.to_sql("SCAC_AT36_ventanahoraria", engine, index=False, if_exists="append", schema="dbo")
